# Game (1 point)
In the following code, you need to complete the `TODO` sections, so that the players using `minimax_search` and `alphabeta_search` can easily defeat the player using `random_player`, while the players using `minimax_search` and `alphabeta_search` can only end in a draw.

### DDL: 22:00, Nov.8st

The practice will be checked in this lab class or the next lab class(before **Nov.8th**) by teachers or SAs. 

#### What will be tested: 
* That you understand every line of your code, not just copy from somewhere 
* That your program compiles correctly
* Correctness of the program logic 
* That the result is obtained in a reasonable time 

#### Grading: 

* Submissions in this lab class: 1.1 points.
* Submissions on time: 1 point.
* Late submissions within 2 weeks after the deadline: 0.8 points.

In [1]:
from game import TicTacToe
import random
import math
import time
infinity = math.inf

# 1. Getting Started with the Game

In [2]:
# initializing a TicTacToe game
game = TicTacToe(height=3, width=3, k=3)
board = game.initial
# get the possible actions
actions = game.actions(board)
# to apply a move (the board automatically figures out which player)
board_after_move = game.result(board, list(actions)[0])

print(board)
print(actions)
print(board_after_move)

. . .
. . .
. . .

{(0, 1), (1, 2), (2, 1), (0, 0), (1, 1), (2, 0), (0, 2), (2, 2), (1, 0)}
. . .
X . .
. . .



In [3]:
def play_game_step(game, state, strategies: dict, verbose=False):
    start = time.perf_counter()
    player = state.to_move
    move = strategies[player](game, state)
    state = game.result(state, move)
    time_elapsed = time.perf_counter() - start
    if verbose: 
        print('Player', player, 'move:', move, f'time: {time_elapsed:.4f}s', )
        print(state)
    return state

def play_game(game, strategies: dict, verbose=False):
    """Play a turn-taking game. `strategies` is a {player_name: function} dict,
    where function(state, game) is used to get the player's move."""
    state = game.initial
    while not game.is_terminal(state):
        state = play_game_step(game, state, strategies, verbose)
    return state

# setup a random strategy for testing
def random_player(game, state): return random.choice(list(game.actions(state)))

def search_player(search_algorithm):
    """A game player who uses the specified search algorithm"""
    return lambda game, state: search_algorithm(game, state)[1] # we expect our search algorithm to return (v, move)

In [4]:
# initialize a game
game = TicTacToe()
state = game.initial
strategies = dict(X=random_player, O=random_player)

In [5]:
# option 1: 
# play_game(game, strategies, True)

# option 2: run it step by step by executing this cell multiple times until the game end
if not game.is_terminal(state):
    state = play_game_step(game, state, strategies, True)
else:
    print(f"Game result: {game.utility(state, 'X')}")
    print(state)

Player X move: (1, 1) time: 0.0000s
. . .
. X .
. . .



# 2. MINIMAX and Alpha-Beta 

### MINIMAX-DECISION and EXPECTIMINIMAX

__function__ MINIMAX-DECISION(_state_) __returns__ _an action_  
&emsp;__return__ arg max<sub> _a_ &Element; ACTIONS(_s_)</sub> MIN\-VALUE(RESULT(_state_, _a_))  

---
__function__ MAX\-VALUE(_state_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __then return__ UTILITY(_state_)  
&emsp;_v_ &larr; &minus;&infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MAX(_v_, MIN\-VALUE(RESULT(_state_, _a_)))  
&emsp;__return__ _v_  

---
__function__ MIN\-VALUE(_state_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __then return__ UTILITY(_state_)  
&emsp;_v_ &larr; &infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MIN(_v_, MAX\-VALUE(RESULT(_state_, _a_)))  
&emsp;__return__ _v_  

---
__function__ EXPECTIMINIMAX(_s_) =     
&emsp;UTILITY(_s_) __if__ TERMINAL\-TEST(_s_)  
&emsp;max<sub>_a_</sub> EXPECTIMINIMAX(RESULT(_s, a_)) __if__ PLAYER(_s_)= MAX  
&emsp;min<sub>_a_</sub> EXPECTIMINIMAX(RESULT(_s, a_)) __if__ PLAYER(_s_)= MIN  
&emsp;∑<sub>_r_</sub> P(_r_) EXPECTIMINIMAX(RESULT(_s, r_)) __if__ PLAYER(_s_)= CHANCE  


In [6]:
def minimax_search(game, state):
    """Search game tree to determine best move; return (value, move) pair."""

    player = state.to_move  # Determine whose turn it is

    def max_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a))
            if v2 > v:
                v, move = v2, a
        return v, move

    def min_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a))
            if v2 < v:
                v, move = v2, a
        return v, move

    # Start by maximizing for the current player
    return max_value(state)

# Test with a TicTacToe game
%time play_game(TicTacToe(), dict(X=search_player(minimax_search), O=random_player), True)


Player X move: (0, 1) time: 7.1061s
. . .
X . .
. . .

Player O move: (1, 0) time: 0.0000s
. O .
X . .
. . .

Player X move: (0, 0) time: 0.0916s
X O .
X . .
. . .

Player O move: (2, 0) time: 0.0000s
X O O
X . .
. . .

Player X move: (1, 2) time: 0.0022s
X O O
X . .
. X .

Player O move: (2, 2) time: 0.0000s
X O O
X . .
. X O

Player X move: (2, 1) time: 0.0001s
X O O
X . X
. X O

Player O move: (0, 2) time: 0.0000s
X O O
X . X
O X O

Player X move: (1, 1) time: 0.0000s
X O O
X X X
O X O

CPU times: total: 7.08 s
Wall time: 7.2 s


X O O
X X X
O X O

In [7]:
# if you have implemented it right, it will always be a draw for h=w=k=3
%time play_game(TicTacToe(), dict(X=search_player(minimax_search), O=search_player(minimax_search)), True)

Player X move: (0, 1) time: 7.2742s
. . .
X . .
. . .

Player O move: (2, 1) time: 0.7940s
. . .
X . O
. . .

Player X move: (1, 2) time: 0.1256s
. . .
X . O
. X .

Player O move: (0, 0) time: 0.0265s
O . .
X . O
. X .

Player X move: (1, 1) time: 0.0050s
O . .
X X O
. X .

Player O move: (1, 0) time: 0.0013s
O O .
X X O
. X .

Player X move: (2, 0) time: 0.0003s
O O X
X X O
. X .

Player O move: (0, 2) time: 0.0001s
O O X
X X O
O X .

Player X move: (2, 2) time: 0.0000s
O O X
X X O
O X X

CPU times: total: 8.03 s
Wall time: 8.23 s


O O X
X X O
O X X

### ALPHA-BETA-SEARCH

__function__ ALPHA-BETA-SEARCH(_state_) __returns__ an action  
&emsp;_v_ &larr; MAX\-VALUE(_state_, &minus;&infin;, &plus;&infin;)  
&emsp;__return__ the _action_ in ACTIONS(_state_) with value _v_  

---
__function__ MAX\-VALUE(_state_, _&alpha;_, _&beta;_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __then return__ UTILITY(_state_)  
&emsp;_v_ &larr; &minus;&infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MAX(_v_, MIN\-VALUE(RESULT(_state_, _a_), _&alpha;_, _&beta;_))  
&emsp;&emsp;&emsp;__if__ _v_ &ge; _&beta;_ __then return__ _v_  
&emsp;&emsp;&emsp;_&alpha;_ &larr; MAX(_&alpha;_, _v_)  
&emsp;__return__ _v_  

---
__function__ MIN\-VALUE(_state_, _&alpha;_, _&beta;_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __then return__ UTILITY(_state_)  
&emsp;_v_ &larr; &plus;&infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MIN(_v_, MAX\-VALUE(RESULT(_state_, _a_), _&alpha;_, _&beta;_))  
&emsp;&emsp;&emsp;__if__ _v_ &le; _&alpha;_ __then return__ _v_  
&emsp;&emsp;&emsp;_&beta;_ &larr; MIN(_&beta;_, _v_)  
&emsp;__return__ _v_  


In [8]:
def alphabeta_search(game, state):
    """Search game to determine best action; use alpha-beta pruning.
    As in [Figure 5.7], this version searches all the way to the leaves."""
    
    player = state.to_move 

    def max_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -float('inf'), None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta)
            if v2 > v:
                v, move = v2, a
            alpha = max(alpha, v)
            if alpha >= beta:
                break  # Beta cutoff
        return v, move

    def min_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = float('inf'), None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            if v2 < v:
                v, move = v2, a
            beta = min(beta, v)
            if alpha >= beta:
                break  # Alpha cutoff
        return v, move

    # Start the alpha-beta pruning search with alpha = -infinity and beta = +infinity
    return max_value(state, -infinity, infinity)

# Test with a TicTacToe game
%time play_game(TicTacToe(), dict(X=search_player(alphabeta_search), O=random_player), True)


Player X move: (0, 1) time: 0.3361s
. . .
X . .
. . .

Player O move: (0, 0) time: 0.0000s
O . .
X . .
. . .

Player X move: (1, 1) time: 0.0163s
O . .
X X .
. . .

Player O move: (0, 2) time: 0.0000s
O . .
X X .
O . .

Player X move: (1, 2) time: 0.0005s
O . .
X X .
O X .

Player O move: (2, 2) time: 0.0000s
O . .
X X .
O X O

Player X move: (1, 0) time: 0.0001s
O X .
X X .
O X O

CPU times: total: 359 ms
Wall time: 353 ms


O X .
X X .
O X O

In [9]:
%time play_game(TicTacToe(), dict(X=search_player(alphabeta_search), O=search_player(alphabeta_search)), True)

Player X move: (0, 1) time: 0.3633s
. . .
X . .
. . .

Player O move: (2, 1) time: 0.0615s
. . .
X . O
. . .

Player X move: (1, 2) time: 0.0148s
. . .
X . O
. X .

Player O move: (0, 0) time: 0.0055s
O . .
X . O
. X .

Player X move: (1, 1) time: 0.0020s
O . .
X X O
. X .

Player O move: (1, 0) time: 0.0002s
O O .
X X O
. X .

Player X move: (2, 0) time: 0.0001s
O O X
X X O
. X .

Player O move: (0, 2) time: 0.0001s
O O X
X X O
O X .

Player X move: (2, 2) time: 0.0000s
O O X
X X O
O X X

CPU times: total: 422 ms
Wall time: 448 ms


O O X
X X O
O X X